# Regresión logística sin librerías:

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [59]:
datos = pd.read_csv('reglog.csv')
datos.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [60]:
# Vamos a cuantificar la dimensión 'Gender' en ceros y unos

datos['Gender'] = datos['Gender'].map({'Male':0, 'Female':1})
datos.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,0,19,19000,0
1,15810944,0,35,20000,0
2,15668575,1,26,43000,0
3,15603246,1,27,57000,0
4,15804002,0,19,76000,0


Podemos intuir que La variable a predecir es "Purchased", pues es binaria (0 o 1) y nos indica si el usuario realizó o no una compra con base en el resto de características

In [61]:
# Dividimos el conjunto de datos en las variables independientes y la dependiente
X = datos[['Gender', 'Age', 'EstimatedSalary']].values
Y = datos['Purchased'].values

In [62]:
# agregamos una columna de unos al principio del conjunto de datos de características 
# para que nos ayuden a multiplicar por uno el intercepto del  modelo y sumarlo a las
# demás características
# y normalizamos

X_media = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X_normalizada = (X - X_media) / X_std

X_intercepto = np.hstack([np.ones((X_normalizada.shape[0], 1)), X])

## 1. Creamos la función logística o sigmoide

In [63]:
# Creamos la función logística
funcion_sigmoide = lambda z : 1 / (1 + np.exp(-z))

##### El resultado es la probabilidad de que y = 1 dado X. 
##### Z es el producto punto entre los coeficientes del modelo y las características de una observación.

In [64]:
# Inicializamos los coeficientes del modelo en ceros:
coeficientes = np.zeros(X_normalizada.shape[1])

## 2. Creamos la función de costo
Esta función va a medir el rendimiento del modelo en términos de qué tan bien las probabilidades predichas coinciden con las clases reales.

La fórmula es:
<img src="costo.png" width="900"/>


In [65]:
# Creamos la función de costo
def funcion_costo(x,y,betas): # x
    m = len(y) # la longitud de 'y'
    
    # Se calcula la predicción para cada observación en el conjunto de datos
    predicciones = funcion_sigmoide(np.dot(x,betas))
    
    # se calculan todos los errores para cada observación de la dimensión 'y'
    error = -y*np.log(predicciones) + (1-y)*np.log(1-predicciones)
    
    # finalmente, se suman todos los errores y se dividen entre el total de datos
    JB = np.sum(error) / m
    
    return JB

## 3. Creamos la función de gradiente descendiente
Este método va a ayudarnos a optimizar, a través de la actualizacón de los parámetros del vector de coeficientes de manera iterativa con el objetivo de minimizar la función de costo.

Las funciones a utilizar son:

<img src="gradiente.png" width="400"/>
<img src="gradiente2.png" width="400"/>


Se entiende que Beta, o los coeficientes, se irán actualizando conforme el gradiente vaya descendiendo. 

In [66]:
def gradiente_desc(x, y, betas, alfa, iteraciones):
    m = len(y)
    costo_historico = []
    for i in range(iteraciones):
        # Calculamos sigmoide con los coeficientes en ceros para inicializar
        predicciones = funcion_sigmoide(np.dot(x, betas))
        
        # Realizamos la diferencia entre los valores predichos y valores reales de 'y'
        diferencia = predicciones - y
        
        # Hacemos el producto de X transpuesta por la diferencia y dividimos entre la cant de datos
        gradiente = np.dot(x.T, diferencia) / m
        
        # Finalmente, actualizamos los valores de los coeficientes Beta como se indica en la fórmula
        betas -= alfa * gradiente
            
        costo_historico.append(funcion_costo(x,y,betas))
        
    return betas

## 4. Comprobación:

In [67]:
alfa = 0.01
iterac = 10000

coeficientes_optimos = gradiente_desc(X_normalizada, Y, coeficientes, alfa, iterac)
coeficientes_optimos

array([-0.07373412,  2.01302946,  1.07226172])

De acuerdo con el modelo, los coeficientes óptimos serían los mostrados arriba, comenzando por el intercepto, y los siguientes son los coeficientes para cada variable independiente.

In [68]:
def prediccion(x, betas):
    return funcion_sigmoide(np.dot(x, betas)) >= 0.5
    

In [69]:
prediccion = prediccion(X_normalizada, coeficientes_optimos)



In [70]:
prediccion[0:20]

array([False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True])

In [71]:
Y[0:20]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
      dtype=int64)